In [1]:
# Import libraries
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

from numpy.polynomial import polynomial as P

In [13]:
# Times are in minutes
stations = [
    #Station Name,          East,   West
    ("Recto",               2.00,   7.00),
    ("Legarda",             2.75,   2.00),
    ("Pureza",              2.75,   2.75),
    ("V. Mapa",             2.50,   2.75),
    ("J. Ruiz",             1.75,   2.50),
    ("Gilmore",             2.25,   1.75),
    ("Betty-Go Belmonte",   2.25,   2.25),
    ("Araneta Cubao",       3.00,   2.25), 
    ("Anonas",              2.00,   3.00),
    ("Katipunan",           4.00,   2.00),
    ("Santolan",            3.50,   4.00),
    ("Marikina-Pasig",      4.50,   3.50),   
    ("Antipolo",            7.00,   4.50)
]

NSTATIONS = len(stations)

[2.0, 2.75, 2.75, 2.5, 1.75, 2.25, 2.25, 3.0, 2.0, 4.0, 3.5, 4.5]

In [ ]:
def read_data(filename):
    """Reads data from a file and returns a numpy array of the data"""
    data = pd.read_csv(filename, header=0)
    return data

monday_raw = read_data("Monday.csv")
monday_cum = monday_raw.cumsum()

monday_cum

,Recto:Entry,Recto:Exit,Legarda:Entry,Legarda:Exit,Pureza:Entry,Pureza:Exit,V. Mapa:Entry,V. Mapa:Exit,J. Ruiz:Entry,J. Ruiz:Exit,...,Anonas:Entry,Anonas:Exit,Katipunan:Entry,Katipunan:Exit,Santolan:Entry,Santolan:Exit,Marikina-Pasig:Entry,Marikina-Pasig:Exit,Antipolo:Entry,Antipolo:Exit
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,139.75,0.00,8.75,0.00,27.25,0.00,22.50,0.00,6.75,0.00,...,28.25,0.00,81.00,0.00,68.50,0.00,165.25,0.00,267.50,0.00
2,931.50,1339.50,108.75,570.50,243.50,227.75,226.25,167.75,87.75,95.50,...,409.00,212.75,807.75,145.00,530.50,105.00,1813.25,126.00,2760.00,113.00
3,2498.25,4405.75,341.50,3330.50,802.00,778.75,881.50,526.75,390.00,335.00,...,1180.50,1371.25,1697.50,780.75,1426.50,329.50,4681.50,488.75,7501.25,378.75
4,4480.50,8058.50,644.75,5537.25,1577.75,1448.25,1667.50,1150.25,669.50,773.00,...,1983.50,3154.75,2705.00,1598.75,2316.25,555.25,7387.50,995.75,11615.75,813.25
5,6122.50,11700.75,869.50,7056.50,2058.25,2238.00,2163.50,1784.00,863.75,1204.25,...,2601.25,4378.75,3742.50,2268.25,2952.50,725.75,9655.00,1495.25,14437.75,1214.00
6,7721.25,14263.75,1410.00,8156.50,2388.50,2944.00,2498.25,2291.25,1005.50,1396.25,...,3110.50,5136.50,4515.50,2726.25,3348.75,873.75,11187.50,1996.00,16519.75,1587.25
7,9274.50,16651.75,2151.50,9169.50,2757.25,3648.50,2811.25,2748.25,1108.00,1531.50,...,3655.25,5732.00,5102.00,3094.25,3641.25,1052.75,12325.50,2629.00,17970.00,2100.75
8,10972.50,18397.25,3032.75,9986.75,3188.25,4177.50,3137.00,3196.50,1216.75,1642.25,...,4221.50,6233.50,5685.00,3453.25,3893.00,1238.75,13421.50,3415.25,19385.50,2728.75
9,12705.00,20284.00,3965.75,11072.50,3706.50,4676.75,3548.50,3620.75,1320.75,1739.75,...,4843.25,6779.25,6302.00,3871.25,4171.00,1480.25,14479.00,4243.75,20584.50,3467.50


In [ ]:
class Station():
    def __init__(self, name, entry_coeffs, exit_coeffs, idx, east_time, west_time):
        self.name = name
        # entry and exit in hours
        self.entry = P.Polynomial(entry_coeffs)
        self.exit = P.Polynomial(exit_coeffs)
        self.idx = idx
        self.east_time = east_time
        self.west_time = west_time

        # next stations
        self.east_next = None
        self.west_next = None

        # last time a train arrived
        self.last_east = 0
        self.last_west = 0

        # number of passengers waiting
        self.east_bound = 0
        self.west_bound = 0

    def step(self, start, step_size):
        # entering passengers
        entering = max(0, self.entry(start + step_size) - self.entry(start))

        self.east_bound += entering * (12 - self.idx) / 12
        self.west_bound += entering * self.idx / 12
        
        # potentially exiting passengers
        exiting = self.exit(start + step_size) - self.exit(start)

        exiting_east = exiting * self.idx / 12
        exiting_west = exiting * (12 - self.idx) / 12
    
    def __str__(self):
        return self.name


class Train():
    def __init__(self, capacity, name, start_station, time_to_active):
        self.capacity = capacity
        self.name = name
        self.time_to_active = time_to_active

        self.passengers = 0
        self.running = False
        self.time_left = 0
        self.east = True
        self.next_station = start_station
    
    def step(self, time, step_size):
        if not self.running:
            self.time_to_active -= step_size

            if self.time_to_active <= 0:
                self.running = True
                print("Train", self.name, "started at", self.next_station.name)

                self.time_left = self.next_station.west_next.east_time
                
            return
        
        self.time_left -= step_size
        print("Train", self.name, "en route to", self.next_station.name, "with", round(self.time_left * 60, 2), "minutes left")
        if self.time_left <= 0.0001:
            curr_station = self.next_station

            print("Train", self.name, "arrived at", curr_station.name)
            if self.east:
                # reverse if terminal station
                if curr_station == curr_station.east_next:
                    self.east = False

                # Unload passengers
                exiting_passengers = (curr_station.exit(time) - curr_station.exit(curr_station.last_east)) * curr_station.idx / 12
                print(curr_station.exit(time), curr_station.exit(curr_station.last_east), curr_station.idx)
                print("Unloading", exiting_passengers, "passengers")
                self.passengers -= exiting_passengers
                curr_station.last_east = time

                # load passengers
                print("Loading", curr_station.east_bound, "passengers")
                self.passengers += curr_station.east_bound
                curr_station.east_bound = 0

                print("Train", self.name, "has", self.passengers, "passengers")

                self.time_left = curr_station.east_time
                self.next_station = curr_station.east_next
                print("Next station", curr_station, "Ang susunod na istasyon ay", curr_station)
                print("Ang oras na natitira ay", round(self.time_left * 60, 2))
            else:
                if curr_station == curr_station.west_next:
                    self.east = True

                self.passengers += curr_station.west_bound
                self.time_left = curr_station.west_time
                self.next_station = curr_station.west_next


class Global():
    def __init__(self, step_size, stations, trains):
        self.step_size = step_size
        self.stations = stations
        self.trains = trains
        self.time = 0

    def step(self):
        print("\nTime: ", round(self.time * 60, 2), "minutes")

        for station in self.stations:
            station.step(self.time, self.step_size)
        
        for train in self.trains:
            train.step(self.time, self.step_size)
        
        self.time += self.step_size

In [ ]:
interpolated = {}

for name, series in monday_cum.items():
    x = monday_cum.index.values
    y = series.values

    poly = P.polyfit(x, y, 10)

    interpolated[name] = poly

print(interpolated)

interpolated_df = pd.DataFrame(list(interpolated.items()), columns=['Name', 'Coefficients'])

interpolated_df

{'Recto:Entry': array([ 4.09312166e+00,  5.63537897e+02, -1.34836472e+03,  1.28644699e+03,
       -4.63718824e+02,  9.15924033e+01, -1.09407403e+01,  8.09030199e-01,
       -3.61079897e-02,  8.88884744e-04, -9.25068033e-06]), 'Recto:Exit': array([-4.11138144e+00,  1.00920542e+03, -2.63564064e+03,  2.26605451e+03,
       -7.24118312e+02,  1.25671951e+02, -1.32388679e+01,  8.71799051e-01,
       -3.50930767e-02,  7.89666054e-04, -7.60680879e-06]), 'Legarda:Entry': array([ 5.59146745e+00, -1.73883474e+02,  1.31949609e+02,  5.71978572e+01,
       -5.60613218e+01,  1.72143847e+01, -2.66700742e+00,  2.32940694e-01,
       -1.16207476e-02,  3.08960857e-04, -3.39670337e-06]), 'Legarda:Exit': array([ 1.92075479e+01,  8.73952315e+02, -2.74097548e+03,  2.43108035e+03,
       -8.40501849e+02,  1.56042941e+02, -1.72855599e+01,  1.17859882e+00,
       -4.85430770e-02,  1.10846036e-03, -1.07784632e-05]), 'Pureza:Entry': array([ 2.32903725e+00,  3.63048915e+02, -8.23889740e+02,  6.57369308e+02,
      

,Name,Coefficients
0,Recto:Entry,"[4.093121663045758, 563.5378966795156, -1348.3..."
1,Recto:Exit,"[-4.111381440882949, 1009.2054196584809, -2635..."
2,Legarda:Entry,"[5.591467452368399, -173.883473528325, 131.949..."
3,Legarda:Exit,"[19.20754794087256, 873.9523147121585, -2740.9..."
4,Pureza:Entry,"[2.3290372498981182, 363.04891525364707, -823...."
5,Pureza:Exit,"[-2.0218619908236266, 240.57158545520866, -562..."
6,V. Mapa:Entry,"[2.593761381518774, 477.98371091551377, -1085...."
7,V. Mapa:Exit,"[-3.3566132416265537, 458.53700744984246, -902..."
8,J. Ruiz:Entry,"[1.6519144081496706, 151.48848591776914, -390...."
9,J. Ruiz:Exit,"[-2.8657101953978725, 412.57777496829704, -772..."


In [ ]:
station_objs = []
train_objs = []

for station in stations:
    entry_poly = interpolated_df[interpolated_df['Name'] == station[0] + ':Entry']['Coefficients'].values[0]
    exit_poly = interpolated_df[interpolated_df['Name'] == station[0] + ':Exit']['Coefficients'].values[0]

    station_instance = Station(station[0], entry_poly, exit_poly, stations.index(station), station[1] / 60, station[2] / 60)

    station_objs.append(station_instance)

station_objs[0].west_next = station_objs[0]
station_objs[-1].east_next = station_objs[-1]

for i in range(len(station_objs) - 1):
    station_objs[i].east_next = station_objs[i + 1]
    station_objs[i + 1].west_next = station_objs[i]

for i in range(10):
    train_objs.append(Train(1000, i, station_objs[0], i * 8 / 60))

print(station_objs[0].west_next)

global_state = Global(0.5/60, station_objs, train_objs)

global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()
global_state.step()

Recto

Time:  0 minutes
Train 0 started at Recto

Time:  0.5 minutes
Train 0 en route to Recto with 1.5 minutes left

Time:  1.0 minutes
Train 0 en route to Recto with 1.0 minutes left

Time:  1.5 minutes
Train 0 en route to Recto with 0.5 minutes left

Time:  2.0 minutes
Train 0 en route to Recto with 0.0 minutes left
Train 0 arrived at Recto
26.68334874286213 -4.111381440882949 0
Unloading 0.0 passengers
Loading 21.231507435006478 passengers
Train 0 has 21.231507435006478 passengers
Next station Recto Ang susunod na istasyon ay Recto
Ang oras na natitira ay 2.0

Time:  2.5 minutes
Train 0 en route to Legarda with 1.5 minutes left

Time:  3.0 minutes
Train 0 en route to Legarda with 1.0 minutes left

Time:  3.5 minutes
Train 0 en route to Legarda with 0.5 minutes left

Time:  4.0 minutes
Train 0 en route to Legarda with 0.0 minutes left
Train 0 arrived at Legarda
65.99284389996342 19.20754794087256 1
Unloading 3.8987746632575715 passengers
Loading 0.0 passengers
Train 0 has 17.3327327